In [5]:
!unzip -q /content/negative.zip -d /content/lfw-dataset


In [6]:
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import os
import random
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Flatten, Dense, Layer
import torch.nn.functional as F
# from torch.nn import Model, Conv2d, Upsample, Flatten, Linear, MaxPool2d


if torch.cuda.is_available():
  print("available GPUS:")
  for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


POS_PATH=os.path.join('data','positive')
NEG_PATH=os.path.join('data','negative')
ANC_PATH=os.path.join('data','anchor')
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

print("Directories created successfully!")

import os
for directory in os.listdir('/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'):
    for file in os.listdir(os.path.join('/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled', directory)):
        EX_PATH = os.path.join('/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

Directories created successfully!


In [7]:

#For some reason this code can only crawl positive images. I needed to create another code for negative iamges
!pip install icrawler Pillow mtcnn

import os  # Added import
from icrawler.builtin import BingImageCrawler
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import numpy as np
import shutil
import uuid

def scrape_and_store_images(person_name, num_images, positive_path, anchor_path):
    # Ensure the save paths exist
    if not os.path.exists(positive_path):
        os.makedirs(positive_path)
        print(f"Created directory: {positive_path}")
    else:
        print(f"Directory already exists: {positive_path}")

    if not os.path.exists(anchor_path):
        os.makedirs(anchor_path)
        print(f"Created directory: {anchor_path}")
    else:
        print(f"Directory already exists: {anchor_path}")

    # Initialize the Bing crawler
    crawler = BingImageCrawler(storage={"root_dir": "temp_images"})
    print(f"Starting image crawl for '{person_name}' with max_num={num_images}")
    crawler.crawl(keyword=person_name, max_num=num_images)  # Download specified number of images
    print("Image crawling completed.")

    # Process each downloaded image
    image_files = [f for f in os.listdir("temp_images") if f.endswith(('jpg', 'jpeg', 'png'))]
    print(f"Number of images downloaded: {len(image_files)}")

    if not image_files:
        raise FileNotFoundError("No images were downloaded. Please refine your search query.")

    detector = MTCNN()
    saved_count = 0
    for idx, image_file in enumerate(image_files):
        image_path = os.path.join("temp_images", image_file)
        try:
            downloaded_image = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB mode
            image_np = np.array(downloaded_image)  # Convert to NumPy array for MTCNN

            # Detect faces
            faces = detector.detect_faces(image_np)
            if faces:
                # Get the first detected face's bounding box
                x, y, width, height = faces[0]["box"]

                # Expand the bounding box by 30%
                expansion_factor = 0.3
                x_expand = int(width * expansion_factor)
                y_expand = int(height * expansion_factor)

                # Calculate the new bounding box coordinates
                x_new = max(0, x - x_expand)
                y_new = max(0, y - y_expand)
                width_new = min(downloaded_image.width, x + width + x_expand) - x_new
                height_new = min(downloaded_image.height, y + height + y_expand) - y_new

                # Crop the expanded face region
                cropped_face = downloaded_image.crop((x_new, y_new, x_new + width_new, y_new + height_new))

                # Pad to maintain aspect ratio
                max_side = max(cropped_face.size)
                padded_face = ImageOps.expand(
                    cropped_face,
                    border=(max_side - cropped_face.width) // 2,
                    fill=(0, 0, 0)
                )

                # Resize to 250x250
                resized_face = padded_face.resize((250, 250))

                # Save the processed image with a unique name
                save_path = positive_path if idx < num_images // 2 else anchor_path  # Split into positive and anchor
                filename = f"{uuid.uuid4().hex}.jpg"
                save_path_full = os.path.join(save_path, filename)
                resized_face.save(save_path_full)

                print(f"Saved: {save_path_full}")
                saved_count += 1

            else:
                print(f"No face detected in {image_file}. Skipping.")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    print(f"Total images saved: {saved_count}")

    # Clean up temporary folder
    shutil.rmtree("temp_images")
    print("Temporary images folder removed.")

# Example usage
POS_PATH = os.path.join('data', 'positive')
ANCHOR_PATH = os.path.join('data', 'anchor')
scrape_and_store_images(
    person_name="Elon Musk",
    num_images=800,
    positive_path=POS_PATH,
    anchor_path=ANC_PATH  # Corrected variable name
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
Directory already exists: data/positive
Directory already exists: data/anchor
Starting image crawl for 'Elon Musk' with max_num=800


ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining r

Image crawling completed.
Number of images downloaded: 283
Saved: data/positive/044faf20c163408c9a382703b6e01fae.jpg
Saved: data/positive/0b84305920284043af767718db151592.jpg
Saved: data/positive/abaa8ebaced9494297c0e777a6b498cb.jpg
Saved: data/positive/d960b5bdf1994613a400998d122b0287.jpg
Saved: data/positive/1441cc10121243bdab3fc84ca7bae495.jpg
Saved: data/positive/51ccda15f7b14e83bd33937780fdb8b6.jpg
Saved: data/positive/ab175452a05c4fe7a2375e22afa08539.jpg
Saved: data/positive/af084815cee84c23885dfaa9df4899b4.jpg
Saved: data/positive/00f40a5cec694eaca41020b6e63a7dc5.jpg
Saved: data/positive/2e2e2a15de044222b64a0cc1996c673a.jpg
Saved: data/positive/8e93fd0625a549f7be74a95e09356a7c.jpg
Saved: data/positive/021dc4e9ab6b4c729a5de557d4d03c5f.jpg
Saved: data/positive/72fca6081d5b43849ebc37a60981fd71.jpg
No face detected in 000265.jpg. Skipping.
Saved: data/positive/6fafe6c696174c439b04e87e538b0099.jpg
Saved: data/positive/14bb7cda68f843629323ca5119bfefab.jpg
Saved: data/positive/9a3f5de0

In [8]:
def scrape_and_store_anchor_images(person_name, num_images, anchor_path):
    """
    Scrapes images of a specified person and saves all processed images to the anchor path.

    Parameters:
    - person_name (str): The name of the person to search for.
    - num_images (int): The number of images to download.
    - anchor_path (str): The directory path where anchor images will be saved.
    """

    # Ensure the anchor directory exists
    if not os.path.exists(anchor_path):
        os.makedirs(anchor_path)
        print(f"Created directory: {anchor_path}")
    else:
        print(f"Directory already exists: {anchor_path}")

    # Initialize the Bing crawler to download images
    crawler = BingImageCrawler(storage={"root_dir": "temp_images"})
    print(f"Starting image crawl for '{person_name}' with max_num={num_images}")
    crawler.crawl(keyword=person_name, max_num=num_images)
    print("Image crawling completed.")

    # List all downloaded image files
    image_files = [f for f in os.listdir("temp_images") if f.lower().endswith(('jpg', 'jpeg', 'png'))]
    print(f"Number of images downloaded: {len(image_files)}")

    if not image_files:
        raise FileNotFoundError("No images were downloaded. Please refine your search query.")

    # Initialize MTCNN for face detection
    detector = MTCNN()
    saved_count = 0

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join("temp_images", image_file)
        try:
            # Open and convert image to RGB
            downloaded_image = Image.open(image_path).convert("RGB")
            image_np = np.array(downloaded_image)

            # Detect faces in the image
            faces = detector.detect_faces(image_np)
            if faces:
                # Get the first detected face's bounding box
                x, y, width, height = faces[0]["box"]

                # Expand the bounding box by 30%
                expansion_factor = 0.3
                x_expand = int(width * expansion_factor)
                y_expand = int(height * expansion_factor)

                # Calculate new bounding box coordinates with expansion
                x_new = max(0, x - x_expand)
                y_new = max(0, y - y_expand)
                width_new = min(downloaded_image.width, x + width + x_expand) - x_new
                height_new = min(downloaded_image.height, y + height + y_expand) - y_new

                # Crop the expanded face region
                cropped_face = downloaded_image.crop((x_new, y_new, x_new + width_new, y_new + height_new))

                # Pad to maintain aspect ratio
                max_side = max(cropped_face.size)
                padded_face = ImageOps.expand(
                    cropped_face,
                    border=(max_side - cropped_face.width) // 2,
                    fill=(0, 0, 0)
                )

                # Resize to 250x250 pixels
                resized_face = padded_face.resize((250, 250))

                # Generate a unique filename and save the image to the anchor path
                filename = f"{uuid.uuid4().hex}.jpg"
                save_path_full = os.path.join(anchor_path, filename)
                resized_face.save(save_path_full)

                # print(f"Saved: {save_path_full}")
                saved_count += 1
            else:
                print(f"No face detected in {image_file}. Skipping.")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    print(f"Total images saved to anchor: {saved_count}")

    # Clean up temporary images
    shutil.rmtree("temp_images")
    print("Temporary images folder removed.")

# Example usage
ANCHOR_PATH = os.path.join('data', 'anchor')
scrape_and_store_anchor_images(
    person_name="Elon Musk",
    num_images=400,  # Adjust as needed
    anchor_path=ANCHOR_PATH
)

Directory already exists: data/anchor
Starting image crawl for 'Elon Musk' with max_num=400


ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining r

Image crawling completed.
Number of images downloaded: 244
No face detected in 000142.jpg. Skipping.
Total images saved to anchor: 243
Temporary images folder removed.


In [9]:
from PIL import Image
from torchvision import transforms
def preprocess(file_path):
  img=Image.open(file_path.convet("RGB"))

  transform=transforms.Compose([
      transforms.Resize((100,100)),
      transform.ToTensor(),
  ])
  img=transforms(img)
  return img

In [55]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import random


class AnchorValidationDataset(Dataset):
    def __init__(self, anchor_paths, validation_paths, labels, transform=None):
        """
        Custom dataset for pairing anchors with validation images (positives and negatives).
        Args:
            anchor_paths (list): List of anchor image paths.
            validation_paths (list): List of validation image paths (positives and negatives).
            labels (list): Corresponding labels for validation images (1 for positive, 0 for negative).
            transform (callable): Transformations to apply to images.
        """
        self.anchor_paths = anchor_paths
        self.validation_paths = validation_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load anchor and validation images
        anchor = Image.open(self.anchor_paths[idx]).convert("RGB")
        validation = Image.open(self.validation_paths[idx]).convert("RGB")

        # Apply transformations
        if self.transform:
            anchor = self.transform(anchor)
            validation = self.transform(validation)

        # Get the label
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        return anchor, validation, label


# Specify the limit and transformations
limit = 100
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),  # Augmentation example
    transforms.ToTensor(),
])

# Anchor and validation image paths
anchor_files = sorted([os.path.join(ANC_PATH, f) for f in os.listdir(ANC_PATH) if f.endswith('.jpg')])
positive_files = sorted([os.path.join(POS_PATH, f) for f in os.listdir(POS_PATH) if f.endswith('.jpg')])
negative_files = sorted([os.path.join(NEG_PATH, f) for f in os.listdir(NEG_PATH) if f.endswith('.jpg')])

# Ensure we have exactly 300 positives and 300 negatives
positive_files = positive_files[:limit]
negative_files = negative_files[:limit]

# Double the anchors to ensure sufficient pairs
anchor_files = anchor_files[:limit] * 2  # Now we have 600 anchors

# Combine validation paths and labels
validation_paths = positive_files + negative_files
labels = [1] * len(positive_files) + [0] * len(negative_files)

# Ensure anchors match validation pairs (300 positive + 300 negative = 600 validation pairs)
if len(anchor_files) != len(validation_paths):
    raise ValueError("Number of anchors must match the total number of validation pairs.")

# Create the dataset
dataset = AnchorValidationDataset(anchor_files, validation_paths, labels, transform=transform)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Iterate through the DataLoader
for batch in dataloader:
    anchor, validation, label = batch
    print(f"Anchor shape: {anchor.shape}, Validation shape: {validation.shape}, Label shape: {label.shape}")


Anchor shape: torch.Size([32, 3, 100, 100]), Validation shape: torch.Size([32, 3, 100, 100]), Label shape: torch.Size([32])
Anchor shape: torch.Size([32, 3, 100, 100]), Validation shape: torch.Size([32, 3, 100, 100]), Label shape: torch.Size([32])
Anchor shape: torch.Size([32, 3, 100, 100]), Validation shape: torch.Size([32, 3, 100, 100]), Label shape: torch.Size([32])
Anchor shape: torch.Size([32, 3, 100, 100]), Validation shape: torch.Size([32, 3, 100, 100]), Label shape: torch.Size([32])
Anchor shape: torch.Size([32, 3, 100, 100]), Validation shape: torch.Size([32, 3, 100, 100]), Label shape: torch.Size([32])
Anchor shape: torch.Size([32, 3, 100, 100]), Validation shape: torch.Size([32, 3, 100, 100]), Label shape: torch.Size([32])
Anchor shape: torch.Size([8, 3, 100, 100]), Validation shape: torch.Size([8, 3, 100, 100]), Label shape: torch.Size([8])


In [13]:
!pip install torchinfo

In [24]:
import torch
import torch.nn as nn
from torchinfo import summary

class Embedding(nn.Module):
    def __init__(self, in_channels=3):
        super(Embedding, self).__init__()
        self.in_channels = in_channels
        self.layers = nn.Sequential(
            # First Layer
            nn.Conv2d(in_channels, out_channels=64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=1),  # Removed padding=1
            # Second Layer
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=7),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=1),  # Removed padding=1
            # Third Layer
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # Removed padding=1
            # Fourth Layer
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),
            nn.ReLU(),
            nn.Flatten()
        )

        # Dynamically calculate the input size for the Linear layer
        with torch.no_grad():
            dummy_input = torch.zeros(1, in_channels, 100, 100)  # Batch size 1, 3 channels, 100x100 image
            flattened_size = self.layers(dummy_input).shape[1]

        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 4096),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layers(x)
        x = self.fc(x)
        return x

# Instantiate the model
model = Embedding()

# Print model summary
summary(model, input_size=(1, 3, 100, 100))  # Batch size 1, 3 channels, 100x100 image


Layer (type:depth-idx)                   Output Shape              Param #
Embedding                                [1, 4096]                 --
├─Sequential: 1-1                        [1, 9216]                 --
│    └─Conv2d: 2-1                       [1, 64, 91, 91]           19,264
│    └─ReLU: 2-2                         [1, 64, 91, 91]           --
│    └─MaxPool2d: 2-3                    [1, 64, 46, 46]           --
│    └─Conv2d: 2-4                       [1, 128, 40, 40]          401,536
│    └─ReLU: 2-5                         [1, 128, 40, 40]          --
│    └─MaxPool2d: 2-6                    [1, 128, 21, 21]          --
│    └─Conv2d: 2-7                       [1, 128, 18, 18]          262,272
│    └─ReLU: 2-8                         [1, 128, 18, 18]          --
│    └─MaxPool2d: 2-9                    [1, 128, 9, 9]            --
│    └─Conv2d: 2-10                      [1, 256, 6, 6]            524,544
│    └─ReLU: 2-11                        [1, 256, 6, 6]           

In [36]:
class Embedding(nn.Module):
  def __init__(self,in_channels=3):
    super(Embedding,self).__init__()
    self.in_channels=in_channels
    self.layers=nn.ModuleList([
        nn.Conv2d(in_channels, out_channels=64, kernel_size=10),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, padding=1),  # Removed padding=1
        # Second Layer
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=7),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, padding=1),  # Removed padding=1
        # Third Layer
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),  # Removed padding=1
        # Fourth Layer
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(256 * 6 * 6,out_features=4096),
        nn.Sigmoid()
    ]
    )
  def forward(self,x):
    for layer in self.layers:
      x=layer(x)
    return x

model = Embedding()

# # Create dummy input
# dummy_input = torch.zeros(1, 3, 100, 100)  # Batch size 1, 3 channels, 100x100 image

# # Forward pass
# output = model(dummy_input)
# print("Output shape:", output.shape)

In [35]:
class L1Dist(nn.Module):
  def __init__(self):
    super(L1Dist,self).__init__()

  def forward(self,input_embedding, validation_embedding):
    return torch.abs(input_embedding-validation_embedding)



In [45]:
class Siamese(nn.Module):
  def __init__(self):
    super(Siamese,self).__init__()
    # self.input_image=input_image
    # self.validation_image=validation_image
    self.siamese_layer=L1Dist()
    self.embedding=Embedding()
    self.classifier=nn.Sequential(
        nn.Linear(4096,1),
        nn.Sigmoid()
    )
  def forward(self, input_image, validation_image):
    distances=self.siamese_layer(self.embedding(input_image),self.embedding(validation_image))
    output=self.classifier(distances)
    return output


In [46]:
model = Siamese()
input_image = torch.randn(1, 3, 100, 100)  # Batch size 1, RGB image 100x100
validation_image = torch.randn(1, 3, 100, 100)
output = model(input_image, validation_image)
print("Output shape:", output.shape)  # Should be [1, 1]

Output shape: torch.Size([1, 1])


In [51]:
import torch.optim as optim
opt=optim.Adam(model.parameters(),lr=1e-4)
binary_cross_loss=nn.BCELoss()


In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),  # Save model weights
    'optimizer_state_dict': opt.state_dict(),  # Save optimizer state
    'epoch': epoch  # Save current epoch
}, './checkpoint.pth')

In [56]:
batch = next(iter(dataloader))
anchor, validation, label = batch
def train_step(model, optimizer, criterion, anchor, validation, label):
    model.train()  # Set model to training mode

    # Forward pass: compute predictions
    output = model(anchor, validation)

    # Calculate loss
    loss = criterion(output, label)

    # Backward pass: compute gradients
    optimizer.zero_grad()
    loss.backward()

    # Update model parameters
    optimizer.step()

    return loss.item()


In [63]:


def save_checkpoint(model, optimizer, epoch, filename="checkpoint.pth"):
    checkpoint = {
        'model_state_dict': model.state_dict(),  # Model weights
        'optimizer_state_dict': optimizer.state_dict(),  # Optimizer state
        'epoch': epoch  # Current epoch
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved at epoch {epoch} to {filename}")




epochs = 10
for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}/{epochs}")
    epoch_loss = 0.0

    for batch in dataloader:
        anchor, validation, label = batch
        label = label.float().unsqueeze(-1)  # BCE expects float labels

        # Train step
        loss = train_step(model, opt, binary_cross_loss, anchor, validation, label)
        epoch_loss += loss

    print(f"Epoch {epoch} Loss: {epoch_loss / len(dataloader):.4f}")

    # Save checkpoint every 2 epochs
    if epoch % 2 == 0:
        save_checkpoint(model, opt, epoch)

Epoch 1/10
Epoch 1 Loss: 0.4099
Epoch 2/10
Epoch 2 Loss: 0.3982
Checkpoint saved at epoch 2 to checkpoint.pth
Epoch 3/10
Epoch 3 Loss: 0.3604
Epoch 4/10
Epoch 4 Loss: 0.2668
Checkpoint saved at epoch 4 to checkpoint.pth
Epoch 5/10
Epoch 5 Loss: 0.3130
Epoch 6/10
Epoch 6 Loss: 0.2245
Checkpoint saved at epoch 6 to checkpoint.pth
Epoch 7/10
Epoch 7 Loss: 0.2006
Epoch 8/10
Epoch 8 Loss: 0.2025
Checkpoint saved at epoch 8 to checkpoint.pth
Epoch 9/10
Epoch 9 Loss: 0.1457
Epoch 10/10
Epoch 10 Loss: 0.1244
Checkpoint saved at epoch 10 to checkpoint.pth
